# Website summariser using ollama locally (Static websites only)



In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

## 1. SCRAPING AND CLEANING THE WEBSITE CONTENT



In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

'''
Short Hinglish Summary

1. headers → website ko bewakoof banake dikhata hai ki hum browser hai, bot nahi.

Website class:
2. URL store karta hai.
3. Page fetch karta hai (requests.get).
4. HTML parse karta hai (BeautifulSoup).
5. Title nikalta hai.
6. Unwanted tags (script, style, images) hataata hai.
7. Pure readable text save karta hai.
        
'''

In [ ]:
yo = Website("https://www.moneycontrol.com/news/business/markets/trade-setup-for-september-12-top-15-things-to-know-before-the-opening-bells-13540563.html")
print(yo.title)
print(yo.text)

## 2. MAKING FUNCTION FOR USER AND SYSTEM PROMPTS

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## 🔑 Core Difference between user_prompt_for() and messages_for()

Soch lo:
User kya bolega? → user_prompt_for()

Ab isko system ke sath package karo → messages_for()

In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## 3. Calling ollama to summarize text


In [ ]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model="llama3.2", messages=messages_for(website))
    return response['message']['content']

In [ ]:
summarize("https://www.moneycontrol.com/news/business/markets/trade-setup-for-september-12-top-15-things-to-know-before-the-opening-bells-13540563.html")

## Optional : A function to display this nicely in the Jupyter output, using markdown

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))
    

In [ ]:
display_summary("https://www.moneycontrol.com/news/business/markets/trade-setup-for-september-12-top-15-things-to-know-before-the-opening-bells-13540563.html")